In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

C:\Users\Cédric Symons\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
T = 6  # Num periods
rep = 1  # Tab number

tabs = {'6-1': '6-periods (1)',
    '6-2': '6-periods (2)',
    '12-1': '12-periods (1)',
    '12-2': '12-periods (2)',
    '24-1': '24-periods (1)',
    '24-2': '24-periods (2)',
    '52-1': '52-periods (1)',
    '52-2': '52-periods (2)',
    '104-1': '104-periods (1)',
    '104-2': '104-periods (2)'}

xls = pd.ExcelFile(r'ULSP-instancesR.xlsx')

df = pd.read_excel(xls, sheet_name=tabs[f'{T}-{rep}'])

# Add zero row and adjust index to align with indicies prompt
df.loc[-1] = [0]*6
df.index = df.index + 1
df = df.sort_index()

# Constants
d = df['Demand Forecast']
f = df['Setup Cost']
c = df['Production cost']
h = df['Holding cost']
b = df['Backlogging cost']

D = [d[i:].sum() for i in range(len(d))]

# Set
H = range(1,T+1)
H_0 = range(T+1)

Min:       $\sum_{t=1}^{T} (f_t y_t + c_t x_t + h_t s_t + b_t r_t)$

Subject to: 

$s_{t-1} + x_t - r_{t-1} = d_t + s_t - r_t$, for all $t = 1,...,T$

$x_t \leq D_{1T} y_t$, for all $t = 1,...,T$

$s_0 = s_T = r_0 =r_T = 0$

$x_t \geq 0$, $s_t \geq 0$, $r_t \geq 0$, $y \in \{0,1\}$

In [3]:
def SI_ULSPwB():
    start_time = time.time()
    m = gp.Model("Uncapacitated_Lot_Sizing_Problem_with_Backlogging")

    # decision variables
    x = m.addVars(T+1, vtype=GRB.INTEGER, name="x")  # x
    s = m.addVars(T+1, vtype=GRB.INTEGER, name="s")  # s
    r = m.addVars(T+1, vtype=GRB.INTEGER, name="r")  # r
    y = m.addVars(T+1, vtype=GRB.BINARY, name="y")  # y

    # objective function
    obj = gp.LinExpr()

    for t in H:
        obj.addTerms([f[t], c[t], h[t], b[t]], [y[t], x[t], s[t], r[t]])

    m.setObjective(obj, GRB.MINIMIZE)

    # constraints
    for t in H:
#         m.addConstr(s[t - 1] + x[t] - r[t - 1] == d[t] + s[t] - r[t], f"inventory_balance_{t}")
        m.addConstr(s[t - 1] + x[t]  == d[t] + s[t] , f"inventory_balance_{t}")
        m.addConstr(x[t] <= D[0] * y[t], f"production_constraint_{t}")
                    
    m.addConstr(s[0] == 0, "initial_inventory")
    m.addConstr(r[0] == 0, "initial_backlog")
    m.addConstr(y[0] == 0)
    m.addConstr(s[T] == 0, "final_inventory")
    m.addConstr(r[T] == 0, "final_backlog")
    # Solve the model
    m.optimize()

    # Print solution
    if m.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print("Period\t\tDemand\tSetup\tProduction\tHolding\tBacklogging\tProduction\tBacklogging\tHolding\tSetup")
        for t in H:
            print(f"Period_{t}\t{d[t]}\t{f[t]}\t{c[t]}\t\t{h[t]}\t{b[t]}\t\t{x[t].X}\t\t{r[t].X}\t\t{s[t].X}\t\t{y[t].X}")
        print(f"Total cost: {m.objVal}")
    else:
        print("No solution found.")
    end_time = time.time()
    total_time = end_time - start_time
    print('Total Time:',total_time)
    x_values = np.zeros(len(H_0))
    y_values = np.zeros(len(H_0))
    s_values = np.zeros(len(H_0))
    r_values = np.zeros(len(H_0))
    for t in H:
        x_values[t] = x[t].X
        y_values[t] = y[t].X
        s_values[t] = s[t].X
        r_values[t] = r[t].X
    return x_values, y_values, s_values, r_values

In [4]:
x, y, s, r = SI_ULSPwB()
print(x, r, s, y)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-07
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 28 columns and 35 nonzeros
Model fingerprint: 0xa3c39bf2
Variable types: 0 continuous, 28 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 6e+02]
Found heuristic solution: objective 106016.00000
Presolve removed 9 rows and 15 columns
Presolve time: 0.02s
Presolved: 8 rows, 13 columns, 20 nonzeros
Variable types: 0 continuous, 13 integer (5 binary)

Root relaxation: objective 5.359542e+04, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work

One of the patterns that can be seen is the model always does the demand of a period in one period; so in other words it won't do half of a period in one period and half in another. This seems normal as it will choose the lowest combination of setup, production/ holding/ backlogging cost.

There will never be holding and backlogging costs in the same period as this would be inprofitable.

Some interesting properties:

$s_{t-1} > 0$ implies $x_t = 0$


$x_t > 0$ implies $r_t = 0$

$s_{t-1} > 0$ implies $r_t = 0$

In [5]:
#TODO: in progress
def SI_ULSPwB_DP():
      
    T = len(d)  # Number of periods
    def D_tu(t, u):
        if t == u:
            return d[t-1]
        return sum(d[t-1:u])
    a_alpha = [0] * (T+1)  
    b_beta = [0] * (T+2)  
    gamma_star =[0] * (T+1)
    beta_star = [0] * (T+1)
    gamma_star = T
    beta_star = T
    
    for alpha, beta in zip(range(T,-1,-1), range(T+1,0,-1)):
        b_betas = []
        for gamma in range(beta, gamma_star+1):
#             print(gamma)
            b_betas.append((sum([h[k-1]*D_tu(k+1, gamma) for k in range(beta, gamma)])+a_alpha[gamma]))

        b_beta[beta] = f[beta-1] + min(b_betas)
        print(b_betas)
        gamma_star = max([beta + index for index, element in enumerate(b_betas) if element == min(b_betas)])
        print('b:', beta, b_beta, 'g*', gamma_star)
        
        a_alphas = []
        for Beta in range(alpha+1, beta_star+1):
#             print(Beta)
            a_alphas.append(sum([b[k-1]*D_tu(alpha+1, k) for k in range(alpha+1, Beta)])+b_beta[Beta])
            print(sum([b[k-1]*D_tu(alpha, k-1) for k in range(alpha+1, Beta)]))


#             print('d:',[D_tu(sigma, k+1)*b[k+1] for k in range(sigma, beta)])
#             print(sum([b[k]*D_tu(sigma, k) for k in range(sigma, beta)]),b_beta[beta+1], sum([b[k]*D_tu(sigma, k) for k in range(sigma, beta)])+b_beta[beta])
        if a_alphas:
            a_alpha[alpha] = min(a_alphas)
        print(a_alphas)
        
        beta_star = max([alpha+1 + index for index, element in enumerate(a_alphas) if element == min(a_alphas)])
        print('a:',alpha, a_alpha, 'b*:', beta_star)
    
    return b_beta[1]


In [6]:
SI_ULSPwB_DP()

KeyError: 7

Min:       $\sum_{t=1}^{T} ()$

Subject to: 

$w_t + $

In [126]:
#TODO: in progress
def SI_ULSPwB_SPR():

    
    def D_ut(u,t):
        return sum(d[u:t+1])
        
    # Create a new model
    m = gp.Model("Uncapacitated_Lot_Sizing_Problem_with_Backlogging_Shortest_Path_Reformulation")
    
    
    # Decision variables
    w = m.addVars(T+1, vtype=GRB.BINARY, name="w")  # Binary variable indicating whether the demand for period t is produced in period t
    phi = m.addVars(T+1, T+1, vtype=GRB.BINARY, name="phi")  # Binary variable = 1 if production in u includes the future demand precisely up to period t ≥ u, and 0 otherwise;
    psi = m.addVars(T+1, T+1, vtype=GRB.BINARY, name="psi")  # Binary variable = 1 if production in u includes backlogged demand precisely from period t ≤ u, and 0 otherwise;
    y = m.addVars(T+1, vtype=GRB.BINARY, name="y")  # y

    # Objective function
    m.setObjective(
        gp.quicksum(f[t] * y[t] +  
                    c[t] * w[t] * d[t] +
                    gp.quicksum(phi[u, t] * (D_ut(u+1, t) * c[u] + gp.quicksum(D_ut(i,t)*h[i] for i in range(u+1, t+1))) for u in range(0, t)) +
                    gp.quicksum(psi[u, t] * (D_ut(t, u-1) * c[u] + gp.quicksum(D_ut(t,i)*b[i] for i in range(t, u))) for u in range(t, T+1))
                    for t in H),
        GRB.MINIMIZE
    )

    # Constraints
    m.addConstr(gp.quicksum(phi[u, T] for u in range(1,T+1)) == 1)
    m.addConstr(gp.quicksum(psi[u, 1] for u in range(1,T+1)) == 1)
    
    m.addConstrs(w[t] <= y[t] for t in range(1,T+1))
    
    for u in H:
        m.addConstr(gp.quicksum(phi[u, t] for t in range(u,T+1)) - w[u] <= 0)
        m.addConstr(gp.quicksum(psi[u, t] for t in range(1,u+1)) - w[u] <= 0)
        
        m.addConstrs(phi[u,t] == 0 for t in range(1,u))
        m.addConstrs(psi[u,t] == 0 for t in range(u+1,T))
    

    m.optimize()

    # Print solution
    if m.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        for v in m.getVars():
            print(f"{v.VarName}: {v.X}")
        print(f"Total cost: {m.objVal}")
    else:
        print("No solution found.")

In [127]:
SI_ULSPwB_SPR()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 45 rows, 112 columns and 103 nonzeros
Model fingerprint: 0xa6114043
Variable types: 0 continuous, 112 integer (112 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+02, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 264434.00000
Presolve removed 31 rows and 94 columns
Presolve time: 0.00s
Presolved: 14 rows, 18 columns, 36 nonzeros
Variable types: 0 continuous, 18 integer (18 binary)
Found heuristic solution: objective 186310.00000

Root relaxation: objective 1.917200e+04, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf